In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data='/kaggle/input/large-license-plate-dataset',
    epochs=75,                            # Number of training epochs
    imgsz=800,                            # Image size
    batch=100,                             # batch size
    lr0=0.01,                             # initial learning rate
    augment=False,                         # no data augmentation
    dropout=0.2                           # dropout to reduce overfitting
)